In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/Infected/omicron/*.csv'
dic_files = Load_files.load_files(path, -14, -12)
data = dic_files['대구']
data

,stdDay,incDec,22D_variants,23A_variants
0,2022-12-19,973.0,467.04,9.73
1,2022-12-20,3625.0,1740.00,36.25
2,2022-12-21,3616.0,1735.68,36.16
3,2022-12-22,3197.0,1534.56,31.97
4,2022-12-23,2657.0,1275.36,26.57
...,...,...,...,...
132,2023-04-30,396.0,75.24,253.44
133,2023-05-01,122.0,23.18,78.08
134,2023-05-02,480.0,91.20,307.20
135,2023-05-03,464.0,88.16,296.96


In [3]:
learning_rates_list = [1e-3, 1e-4]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]
criterions_list = [nn.MSELoss(), criterion2, criterion3]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        for criterion in criterions_list:
                            hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout, criterion]
                            i += 1
                        
print(i)

216


In [4]:
input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', '22D_variants')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [5]:
best_model = {}
models_list = ['seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
               'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), 
                               desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        criterion = hyperparameter[6]
        #print('-------------------------------------------------------------------------------------')
        #print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
        #      ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 62, 60, 7, batch_size, 'mtm')
        
        if num_model == 0:
            model = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)
            
        elif num_model == 2:
            model = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        elif num_model == 4:
            model = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

        elif num_model == 5:
            model = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_Many(train_loader, 
                                                       test_loader, 
                                                       model, 
                                                       criterion, 
                                                       optimizer, 
                                                       num_epochs, 
                                                       patience,
                                                       device)
    
        label_y, predicted, first_label_y, first_predicted = predict_mtm(model, df, x_ss, y_ms, 
                                                                         69, 7, 0.5, device)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout, criterion]

대구_seq2seq_RNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_LSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_GRU:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiRNN:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiLSTM:   0%|          | 0/216 [00:00<?, ?it/s]

대구_seq2seq_BiGRU:   0%|          | 0/216 [00:00<?, ?it/s]

In [6]:
best_model

{'seq2seq_RNN': [0.007681478814780285,
  RNN_encoder_decoder(
    (RNN_encoder): RNN_encoder(
      (rnn): RNN(3, 32, batch_first=True, dropout=0.25)
    )
    (RNN_decoder): RNN_decoder(
      (rnn): RNN(3, 32, batch_first=True, dropout=0.25)
      (linear): Linear(in_features=32, out_features=1, bias=True)
    )
  ),
  [0.0986977107822895,
   0.09416401945054531,
   0.08657377026975155,
   0.0783454142510891,
   0.07690617814660072,
   0.07242433913052082,
   0.06707819737493992,
   0.06513633206486702,
   0.062210872769355774,
   0.06276795081794262,
   0.06326667033135891,
   0.06388535350561142,
   0.060286134481430054,
   0.06073601543903351,
   0.05951235815882683,
   0.060496460646390915,
   0.05955693684518337,
   0.05839823558926582,
   0.059287846088409424,
   0.06066015362739563,
   0.05962236039340496,
   0.05903512425720692,
   0.05715301260352135,
   0.059557920321822166,
   0.05502515472471714,
   0.05706421844661236,
   0.055940425023436546,
   0.057525984942913055,
  

In [7]:
for key, values in best_model.items():
    save_model(values[1].state_dict(), f"model/Infected/22D/{key}.pth")
    save_hyperparameter(values[3:], f"hyperparameter/Infected/22D/{key}.pkl")